In [6]:
import os
import shutil
import pandas as pd
import numpy as np

## Two inputs

In [29]:
input1 = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\xie.xlsx'
input2 = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\tao.xlsx'
#input3 = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\liang.xlsx'

output_folder = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result'

df1 = pd.read_excel(input1)
df2 = pd.read_excel(input2)
#df3 = pd.read_excel(input3)

df1.head(3)



,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1002088,1,4,1,5,1,1,5,1,1,1.0,0.0,0.0,anning
1,1002469,1,4,1,5,1,1,3,3,1,1.0,0.0,0.5,anning
2,1003152,1,1,5,1,1,5,1,1,5,2.0,2.0,2.0,anning


In [5]:
reader1_folder = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\reader_results\derived\huashan_xieyuxue'
reader2_folder = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\reader_results\derived\huashan_taohongyue'
output_folder = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result'

def get_gt(value1, value2):
    diff = abs(value1 - value2)
    if diff == 0:
        gt = value1
    elif diff == 0.5:
        if str(value1).endswith('5'):
            gt = value2
        else:
            gt = value1
    else:
        gt = 0 - diff
    return gt

cols = ['PID', 'GT_Supraspinatus', 'GT_Infraspinatus', 'GT_Subscapularis', 'Xie_Supraspinatus', 'Tao_Supraspinatus', 'Xie_Infraspinatus', 'Tao_Infraspinatus', 'Xie_Subscapularis', 'Tao_Subscapularis', 'Institution']
gt_info = dict()
for col in cols:
    gt_info.setdefault(col, [])

institutions = ['anning', 'huashan_ge', 'huashan_siemens', 'huashan_toshiba', 'huashan_uih', 'qingdao']
for institution in institutions:
    cur_file = institution +'_case_list.xlsx'
    reader1_file = os.path.join(reader1_folder, cur_file)
    reader2_file = os.path.join(reader2_folder, cur_file)
    df1 = pd.read_excel(reader1_file)
    df2 = pd.read_excel(reader2_file)

    
    for i in range(len(df1)):
        gt_info['PID'].append(df1.iloc[i][0])
        supra_1, infra_1, subs_1 = df1.iloc[i][10:13]
        supra_2, infra_2, subs_2 = df2.iloc[i][10:13]
        gt_supra = get_gt(supra_1, supra_2)
        gt_info['GT_Supraspinatus'].append(gt_supra)
        gt_infra = get_gt(infra_1, infra_2)
        gt_info['GT_Infraspinatus'].append(gt_infra)
        gt_subs = get_gt(subs_1, subs_2)
        gt_info['GT_Subscapularis'].append(gt_subs)
        gt_info['Xie_Supraspinatus'].append(supra_1)
        gt_info['Tao_Supraspinatus'].append(supra_2)
        gt_info['Xie_Infraspinatus'].append(infra_1)
        gt_info['Tao_Infraspinatus'].append(infra_2)
        gt_info['Xie_Subscapularis'].append(subs_1)
        gt_info['Tao_Subscapularis'].append(subs_2)
        gt_info['Institution'].append(institution)

gt_df = pd.DataFrame(gt_info)
gt_df.to_excel(os.path.join(output_folder, 'GT_Xie_Tao.xlsx'), index=False)

## Three inputs

In [ ]:
input1 = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\xie.xlsx'
input2 = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\tao.xlsx'
input3 = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\liang.xlsx'

In [81]:
def get_gt2(results_list):
    sorted_results = sorted(results_list)
    diff = sorted_results[2] - sorted_results[0]
    unique_count = len(set(sorted_results))

    if unique_count == 1:
        gt = sorted_results[1]
    elif unique_count == 2:
        if diff == 0.5:
            if str(sorted_results[0]).endswith('0'):
                gt = sorted_results[0]
            else:
                gt = sorted_results[2]
        else:
            gt = sorted_results[1]
    else:
        if diff == 1:
            if str(sorted_results[1]).endswith('0'):
                gt = sorted_results[1]
            else:
                gt = 0 - diff
        elif diff == 1.5:
            gt = 1
        else:
            if sorted_results[1] != 1:
                if sorted_results[1] == 1.5:
                    gt = 2
                else:
                    gt = 0
            else:
                gt = 0- diff
    return gt

def get_gt3(results_list):    
    rank = [0,0,0]    
    for result in results_list:
        if str(result).endswith('0'):
            rank[int(result)] += 1
        else:
            if result == 0.5:
                rank[0] += 0.5
                rank[1] += 0.5
            else:
                rank[1] += 0.5
                rank[2] += 0.5
    max_rank = max(rank)
    min_rank = min(rank)
    if max_rank >= 2:
        gt = rank.index(max_rank)
    elif max_rank == 1.5:
        if not min_rank == 0:
            gt = rank.index(max_rank)
        else:
            min_index = rank.index(min_rank)
            if min_index == 0:
                gt = 12
            elif min_index == 1:
                gt = 20
            else:
                gt = 10
    else:
        gt = 120
    return gt
            
cols = ['PID', 'GT_Supraspinatus', 'GT_Infraspinatus', 'GT_Subscapularis', 'Xie_Supraspinatus', 'Tao_Supraspinatus', 'Liang_Supraspinatus','Xie_Infraspinatus', 'Tao_Infraspinatus','Liang_Infraspinatus', 'Xie_Subscapularis', 'Tao_Subscapularis', 'Liang_Subscapularis','Institution']
gt_info = dict()
for col in cols:
    gt_info.setdefault(col, [])

for i in range(len(df1)):
    gt_info['PID'].append(df1.iloc[i][0])
    supra_1, infra_1, subs_1 = df1.iloc[i][10:13]
    supra_2, infra_2, subs_2 = df2.iloc[i][10:13]
    supra_3, infra_3, subs_3 = df3.iloc[i][10:13]
    gt_supra = get_gt3([supra_1, supra_2, supra_3])
    gt_info['GT_Supraspinatus'].append(gt_supra)
    gt_infra = get_gt3([infra_1, infra_2, infra_3])
    gt_info['GT_Infraspinatus'].append(gt_infra)
    gt_subs = get_gt3([subs_1, subs_2, subs_3])
    gt_info['GT_Subscapularis'].append(gt_subs)
    gt_info['Xie_Supraspinatus'].append(supra_1)
    gt_info['Tao_Supraspinatus'].append(supra_2)
    gt_info['Liang_Supraspinatus'].append(supra_3)
    gt_info['Xie_Infraspinatus'].append(infra_1)
    gt_info['Tao_Infraspinatus'].append(infra_2)
    gt_info['Liang_Infraspinatus'].append(infra_3)
    gt_info['Xie_Subscapularis'].append(subs_1)
    gt_info['Tao_Subscapularis'].append(subs_2)
    gt_info['Liang_Subscapularis'].append(subs_3)
    gt_info['Institution'].append(df1.iloc[i][13])

gt_df = pd.DataFrame(gt_info)
gt_df.to_excel(os.path.join(output_folder, 'GT_Xie_Tao_Liang.xlsx'), index=False)

In [83]:

for tendon in ['Supraspinatus', 'Infraspinatus', 'Subscapularis']:
    input_gt = rf'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\GT_{tendon}_Xie_Tao_Liang.xlsx'
    gt = []
    df = pd.read_excel(input_gt)
    for index, row in df.iterrows():
        gt_result = row[f'GT_{tendon}']
        if len(str(gt_result)) == 2:
            if int(row[f'Xie_{tendon}']) == int(row[f'Tao_{tendon}']):
                gt.append(int(row[f'Xie_{tendon}']))
                continue
        gt.append(gt_result)
    df[f'GT_{tendon}_2'] = gt
    df.to_excel(input_gt, index=False)

## Get cases

In [89]:
input_gt = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\GT_Xie_Tao_Liang.xlsx'
df = pd.read_excel(input_gt)
case_index = []
for index, row in df.iterrows():
    if len(str(row['GT_Supraspinatus_2'])) + len(str(row['GT_Infraspinatus_2'])) + len(str(row['GT_Subscapularis_2'])) > 3:
        case_index.append(index)
df2 = df.iloc[case_index]
df2.to_excel(os.path.join(output_folder,'adj_cases.xlsx'), index=False)


In [93]:
import shutil

case_output = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\adj_cases'
mapping_addr = {'anning': r'F:\huashan\filter_data\anning', 'qingdao': r'F:\huashan\filter_data\qingdao', 'huashan_siemens':r'F:\huashan\filter_data\huashan\SIEMENS', 'huashan_ge': r'F:\huashan\filter_data\huashan\GE', 'huashan_toshiba': r'F:\huashan\filter_data\huashan\TOSHIBA', 'huashan_uih': r'F:\huashan\filter_data\huashan\UIH'}

for index,row in df2.iterrows():
    pid = str(row['PID'])
    source_root = mapping_addr.get(row['Institution'])
    source_folder_addr = os.path.join(source_root, pid)
    dest_root = os.path.join(case_output, row['Institution'])
    dest_folder = '_'.join([row['PID'], str(row['GT_Supraspinatus_2']), str(row['GT_Infraspinatus_2']), str(row['GT_Subscapularis'])])  
    dest_folder_addr = os.path.join(dest_root, dest_folder)
    shutil.copytree(source_folder_addr, dest_folder_addr)


In [4]:
adj_folder = r'F:\huashan\adj_data'
input_gt = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\gt_result\GT_Xie_Tao.xlsx'
gt_df = pd.read_excel(input_gt)
gt_df.head(3)

,PID,GT_Supraspinatus,GT_Infraspinatus,GT_Subscapularis,Xie_Supraspinatus,Tao_Supraspinatus,Xie_Infraspinatus,Tao_Infraspinatus,Xie_Subscapularis,Tao_Subscapularis,Institution,Age,Gender,Manufacturer,Model,Strength
0,1002088,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,1.0,anning,072Y,F,SIEMENS,MAGNETOM_ESSENZA,1.5
1,1002469,-1.0,0.0,1.0,1.0,2.0,0.0,0.0,0.5,1.0,anning,057Y,F,SIEMENS,MAGNETOM_ESSENZA,1.5
2,1003152,2.0,-1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,anning,079Y,F,SIEMENS,MAGNETOM_ESSENZA,1.5


In [18]:
huashan_ge_list = os.listdir(r'F:\huashan\filter_data\huashan\GE')
huashan_siemens_list = os.listdir(r'F:\huashan\filter_data\huashan\SIEMENS')
huashan_toshiba_list = os.listdir(r'F:\huashan\filter_data\huashan\TOSHIBA')
huashan_uih_list = os.listdir(r'F:\huashan\filter_data\huashan\UIH')


for index, row in gt_df.iterrows():
    if row.GT_Supraspinatus < -1:
        if row.Institution == 'anning':
            case_root = r'F:\huashan\preprocessed_data'
        elif row.Institution == 'qingdao':
            case_root = r'F:\huashan\filter_data\qingdao'
        elif row.PID in huashan_ge_list:
            case_root = r'F:\huashan\filter_data\huashan\GE'
        elif row.PID in huashan_siemens_list:
            case_root = r'F:\huashan\filter_data\huashan\SIEMENS'
        elif row.PID in huashan_toshiba_list:
            case_root = r'F:\huashan\filter_data\huashan\TOSHIBA'
        elif row.PID in huashan_uih_list:
            case_root = r'F:\huashan\filter_data\huashan\UIH'
        case_addr = os.path.join(case_root, row.PID)
        output_case_folder = row.PID + '_' + str(row.Xie_Supraspinatus) + '_' + str(row.Tao_Supraspinatus)
        shutil.copytree(case_addr, os.path.join(adj_folder, 'Supraspinatus', output_case_folder))
    if row.GT_Infraspinatus < -1:
        if row.Institution == 'anning':
            case_root = r'F:\huashan\preprocessed_data'
        elif row.Institution == 'qingdao':
            case_root = r'F:\huashan\filter_data\qingdao'
        elif row.PID in huashan_ge_list:
            case_root = r'F:\huashan\filter_data\huashan\GE'
        elif row.PID in huashan_siemens_list:
            case_root = r'F:\huashan\filter_data\huashan\SIEMENS'
        elif row.PID in huashan_toshiba_list:
            case_root = r'F:\huashan\filter_data\huashan\TOSHIBA'
        elif row.PID in huashan_uih_list:
            case_root = r'F:\huashan\filter_data\huashan\UIH'
        output_case_folder = row.PID + '_' + str(row.Xie_Infraspinatus) + '_' + str(row.Tao_Infraspinatus)
        shutil.copytree(case_addr, os.path.join(adj_folder, 'Infraspinatus', output_case_folder))
    if row.GT_Subscapularis < -1:
        if row.Institution == 'anning':
            case_root = r'F:\huashan\preprocessed_data'
        elif row.Institution == 'qingdao':
            case_root = r'F:\huashan\filter_data\qingdao'
        elif row.PID in huashan_ge_list:
            case_root = r'F:\huashan\filter_data\huashan\GE'
        elif row.PID in huashan_siemens_list:
            case_root = r'F:\huashan\filter_data\huashan\SIEMENS'
        elif row.PID in huashan_toshiba_list:
            case_root = r'F:\huashan\filter_data\huashan\TOSHIBA'
        elif row.PID in huashan_uih_list:
            case_root = r'F:\huashan\filter_data\huashan\UIH'
        output_case_folder = row.PID + '_' + str(row.Xie_Subscapularis) + '_' + str(row.Tao_Subscapularis)
        shutil.copytree(case_addr, os.path.join(adj_folder,'Subscapularis', output_case_folder))